In [1]:
import psycopg2
import sys

## below creates the table and populates it with csv file data (pulled tweets)

In [32]:
con = None

create_stmnt = "CREATE TABLE la_tweets(Id bigint PRIMARY KEY, username varchar NOT NULL, user_followers_count integer, user_tweet_count integer, tweet varchar, rt_status smallint, rt_count integer, rt_tweet varchar, fav_count integer);"
insrt_stmnt = "COPY la_tweets (Id, username, user_followers_count, user_tweet_count, tweet, rt_status, rt_count, rt_tweet, fav_count) FROM '/Users/andric/Documents/workspace/insight/twitter_loc/tweets_batch%d.csv' WITH DELIMITER ',' CSV;"

try:
     
    con = psycopg2.connect("dbname='tweetdat' user='andric'")       
    cur = con.cursor()
    
    cur.execute(create_stmnt)
    for i in range(1, 5):
        cur.execute(insrt_stmnt % i)
    con.commit()

except psycopg2.DatabaseError, e:
    
    if con:
        con.rollback()
    
    print 'Error %s' % e    
    sys.exit(1)

finally:

    if con:
        con.close()

In [33]:
con = None

try:
     
    con = psycopg2.connect("dbname='tweetdat' user='andric'")       
    cur = con.cursor()
    
    cur.execute("SELECT * FROM la_tweets LIMIT 3")
    rows = cur.fetchall()

except psycopg2.DatabaseError, e:
    
    if con:
        con.rollback()
    
    print 'Error %s' % e    
    sys.exit(1)

finally:

    if con:
        con.close()

In [34]:
import numpy as np
import pandas as pd

In [39]:
columnnames = ['id', 'username', 'user_followers_count',
               'user_tweet_count', 'tweet', 'rt_status',
               'rt_count', 'rt_tweet', 'fav_count']
df = pd.DataFrame(rows, columns=columnnames)

In [37]:
print df.iloc[0, :]

0                                   872311338382618625
1                                            itsAllaKe
2                                                   95
3                                                 4469
4    RT @WellieBoyce: 3 of them have been arrested....
5                                                    1
6                                                 2310
7    3 of them have been arrested. Thank you. https...
8                                                    0
Name: 0, dtype: object


In [41]:
df

,id,username,user_followers_count,user_tweet_count,tweet,rt_status,rt_count,rt_tweet,fav_count
0,872311338382618625,itsAllaKe,95,4469,RT @WellieBoyce: 3 of them have been arrested....,1,2310,3 of them have been arrested. Thank you. https...,0
1,872311338340634625,alyram1018,424,6613,RT @daisymarquez_: Views ☁️ https://t.co/xReWR...,1,259,Views ☁️ https://t.co/xReWRqj0mW,0
2,872311338206363648,YogirlViv77,261,11608,@runLALArunn lol I don't even watch tv 😂😂,0,0,NO_RT,0


In [104]:
from textblob import TextBlob
import math

In [107]:
df.rt_tweet[0]

'3 of them have been arrested. Thank you. https://t.co/KlxvDGntCx'

In [122]:
analysis = TextBlob('Corrupt Hillary. Sad!')

In [128]:
#wrdlist = TextBlob(df.rt_tweet[0]).split()
wrds = TextBlob(' '.join(df.rt_tweet[0].split()[:-1]))
wrds.sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [162]:
#TextBlob(df.tweet[1].decode('ascii', 'ignore')).sentiment
TextBlob(df.tweet[1].decode('utf-8', 'ignore')).sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [210]:
print df.tweet[1].decode('ascii', 'ignore').lower()

rt @daisymarquez_: views  https://t.co/xrewrqj0mw


In [156]:
import nltk
from nltk.tokenize import RegexpTokenizer

In [204]:
re_tokenizer = RegexpTokenizer(r'\w+')

In [217]:
re_tokenizer.tokenize(df.tweet[1].decode('ascii', 'ignore').lower())

[u'rt', u'daisymarquez_', u'views', u'https', u't', u'co', u'xrewrqj0mw']

In [222]:
df.tweet[1].decode('ascii', 'ignore').lower().split()

[u'rt', u'@daisymarquez_:', u'views', u'https://t.co/xrewrqj0mw']

In [186]:
con = psycopg2.connect("dbname='tweetdat' user='andric'")
sql_query = """
            SELECT * FROM la_tweets LIMIT 7;
            """
query_results = pd.read_sql_query(sql_query, con)
if con:
    con.close()

In [187]:
query_results

,id,username,user_followers_count,user_tweet_count,tweet,rt_status,rt_count,rt_tweet,fav_count
0,872311338382618625,itsAllaKe,95,4469,RT @WellieBoyce: 3 of them have been arrested....,1,2310,3 of them have been arrested. Thank you. https...,0
1,872311338340634625,alyram1018,424,6613,RT @daisymarquez_: Views ☁️ https://t.co/xReWR...,1,259,Views ☁️ https://t.co/xReWRqj0mW,0
2,872311338206363648,YogirlViv77,261,11608,@runLALArunn lol I don't even watch tv 😂😂,0,0,NO_RT,0
3,872311338151927809,MidniteNRG,38,218,https://t.co/3fli08Y918,0,0,NO_RT,0
4,872311338130956288,HollywoodVeal,679,12068,@md0tbrown You sayi my this as a Kings fan or ...,0,0,NO_RT,0
5,872311338097422336,_xokassy,1024,52179,@bnh_xo AWW THANKS BABY !! 😘,0,0,NO_RT,0
6,872311337858330624,brito_jamieBX19,2101,46680,RT @WellieBoyce: I want a fucking manhunt to e...,1,81804,I want a fucking manhunt to expose these kids....,0


In [178]:
resy = ""
for i in range(3):
    resy += query_results.iloc[i]['tweet']
    resy += "<br>"

In [203]:
print query_results.iloc[1]['tweet'].decode('ascii', 'ignore')

RT @daisymarquez_: Views  https://t.co/xReWRqj0mW


## below makes lv_tweets table and populates it

In [9]:
con = None

create_stmnt = "CREATE TABLE lv_tweets(Id bigint PRIMARY KEY, username varchar NOT NULL, user_followers_count integer, user_tweet_count integer, tweet varchar, rt_status smallint, rt_count integer, rt_tweet varchar, fav_count integer);"
insrt_stmnt = "COPY lv_tweets (Id, username, user_followers_count, user_tweet_count, tweet, rt_status, rt_count, rt_tweet, fav_count) FROM '/Users/andric/Documents/workspace/insight/twitter_loc/tweets_lv%d.csv' WITH DELIMITER ',' CSV;"

try:
     
    con = psycopg2.connect("dbname='tweetdat' user='andric'")       
    cur = con.cursor()
    
    cur.execute(create_stmnt)
    for i in range(3, 10):
        print i
        cur.execute(insrt_stmnt % i)
    con.commit()

except psycopg2.DatabaseError, e:
    
    if con:
        con.rollback()
    
    print 'Error %s' % e    
    sys.exit(1)

finally:

    if con:
        con.close()

3
4
5
6
Error duplicate key value violates unique constraint "lv_tweets_pkey"
DETAIL:  Key (id)=(874720208967143425) already exists.
CONTEXT:  COPY lv_tweets, line 132



SystemExit: 1

## add more data to la_tweets table

In [11]:
con = None

insrt_stmnt = "COPY la_tweets (Id, username, user_followers_count, user_tweet_count, tweet, rt_status, rt_count, rt_tweet, fav_count) FROM '/Users/andric/Documents/workspace/insight/twitter_loc/tweets_la%d.csv' WITH DELIMITER ',' CSV;"

try:
     
    con = psycopg2.connect("dbname='tweetdat' user='andric'")       
    cur = con.cursor()
    
    for i in range(110, 111):
        print i
        cur.execute(insrt_stmnt % i)
    con.commit()

except psycopg2.DatabaseError, e:
    
    if con:
        con.rollback()
    
    print 'Error %s' % e    
    sys.exit(1)

finally:

    if con:
        con.close()

110


## Create two more tables (chi and houston)

In [14]:
con = None


create_stmnt = "CREATE TABLE %s_tweets(Id bigint PRIMARY KEY, username varchar NOT NULL, user_followers_count integer, user_tweet_count integer, tweet varchar, rt_status smallint, rt_count integer, rt_tweet varchar, fav_count integer);"

try:

    con = psycopg2.connect("dbname='tweetdat' user='andric'")       
    cur = con.cursor()

    for city in ['chi', 'hou']:
        cur.execute(create_stmnt % city)
    con.commit()

except psycopg2.DatabaseError, e:

    if con:
        con.rollback()

    print 'Error %s' % e    
    sys.exit(1)

finally:

    if con:
        con.close()

In [20]:
con = None

insrt_stmnt = "COPY %s_tweets (Id, username, user_followers_count, user_tweet_count, tweet, rt_status, rt_count, rt_tweet, fav_count) FROM '/Users/andric/Documents/workspace/insight/twitter_loc/tweets_csvs/tweets_%s%d.csv' WITH DELIMITER ',' CSV;"

try:
     
    con = psycopg2.connect("dbname='tweetdat' user='andric'")       
    cur = con.cursor()
    
#    for i in range(15, 27):
    for i in range(25, 27):
        for city in ['la']:
            print i
            cur.execute(insrt_stmnt % (city, city, i))
    con.commit()

except psycopg2.DatabaseError, e:
    
    if con:
        con.rollback()
    
    print 'Error %s' % e    
    sys.exit(1)

finally:

    if con:
        con.close()

25
26
